In [1]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

# Define arrival rates for specific time slots (in cars per minute)
time_slots = [
    (8 * 60, 11 * 60 + 59, 15 / 60),   # 08:00 - 11:59 (15 cars/hour)
    (12 * 60, 16 * 60 + 59, 50 / 60),  # 12:00 - 16:59 (50 cars/hour)
    (17 * 60, 20 * 60, 50 / 60)        # 17:00 - 20:00 (50 cars/hour)
]

# Define simulation parameters
PARKING_SPACES = 100          # Number of parking spaces in the lot
SIMULATION_TIME = 12 * 60 * 60  # Simulation time for 12 hours
PARKING_DURATION_MEAN = 59     # Average time a car spends parked (in minutes)

# Data storage
arrival_times = {i: [] for i in range(len(time_slots))}
departure_times = []
lost_customers = 0
parking_lot_occupancy = []

# New lists to track waiting times and queue lengths
waiting_times = []             # To store the waiting times for each car
queue_lengths = []             # To store the length of the queue over time

# Factors for adjusting lambda based on weather and price
base_wtv = 80                 # Base WTV (neutral)
price_sensitivity = -1.27     # Sensitivity of WTV to price changes (negative: higher price worsens WTV)
base_price = 25               # Base parking price
current_price = 15            # Current price (can adjust to simulate different scenarios)

def calculate_wtv(current_price):
    """Calculate WTV based on the current price and sensitivity."""
    return base_wtv + price_sensitivity * (current_price - base_price)

def adjust_lambda(base_lambda, wtv):
    """Adjust base lambda using the calculated WTV."""
    return base_lambda * (wtv / base_wtv)

def get_current_lambda(current_time):
    """Determine the current lambda based on the simulated time and adjust with weather factors."""
    wtv = calculate_wtv(current_price)  # Calculate current WTV based on price
    for i, (start, end, rate) in enumerate(time_slots):
        if start <= current_time <= end:
            adjusted_rate = adjust_lambda(rate, wtv)
            return adjusted_rate, i
    return 0, -1  # If outside defined time slots, assume no arrivals

def car(env, parking_lot, name, slot):
    global lost_customers
    arrival_time = env.now
    if len(parking_lot.queue) + parking_lot.count < PARKING_SPACES:
        arrival_times[slot].append(arrival_time)
        with parking_lot.request() as request:
            queue_length_at_request = len(parking_lot.queue)  # Track the queue length when car arrives
            start_wait = env.now  # Record the time when car starts waiting
            yield request
            end_wait = env.now  # Record the time when car gets the parking space
            waiting_time = end_wait - start_wait
            waiting_times.append(waiting_time)  # Store the waiting time for this car
            queue_lengths.append(queue_length_at_request)  # Store the queue length when the car started waiting
            # Record parking lot occupancy before and after parking
            parking_lot_occupancy.append((env.now, parking_lot.count))
            parking_duration = np.random.exponential(PARKING_DURATION_MEAN)
            yield env.timeout(parking_duration)
            departure_time = env.now
            departure_times.append(departure_time)
            parking_lot_occupancy.append((env.now, parking_lot.count - 1))
    else:
        lost_customers += 1

def car_generator(env, parking_lot):
    """Generate cars based on the time slots and arrival rates."""
    while env.now < SIMULATION_TIME:
        current_lambda, slot = get_current_lambda(env.now)
        if current_lambda > 0:
            inter_arrival_time = np.random.exponential(1 / current_lambda)
            inter_arrival_time = max(inter_arrival_time, 0.1)  # Avoid generating cars too fast
            yield env.timeout(inter_arrival_time)
            env.process(car(env, parking_lot, f'Car at time {env.now}', slot))
        else:
            yield env.timeout(1)  # Skip time where no cars should arrive

# Set up the environment and the parking lot resource
env = simpy.Environment()
parking_lot = simpy.Resource(env, capacity=PARKING_SPACES)

# Start the car arrival process
env.process(car_generator(env, parking_lot))

# Run the simulation
env.run(until=SIMULATION_TIME)  # Run for the defined simulation time

# Calculate Tq: the average waiting time in queue
Tq = np.mean(waiting_times) if waiting_times else 0
# Calculate Nq: the average number of cars in queue
Nq = np.mean(queue_lengths) if queue_lengths else 0

print(f'Average Waiting Time in Queue (Tq): {Tq:.2f} minutes')
print(f'Average Number of Cars in Queue (Nq): {Nq:.2f}')
print(f'Number of lost customers: {lost_customers}')


Average Waiting Time in Queue (Tq): 0.00 minutes
Average Number of Cars in Queue (Nq): 0.00
Number of lost customers: 0
